In [3]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import scipy.optimize as so
import matplotlib.pyplot as plt
from util import simulate_austin
import importlib

In [2]:
importlib.reload(simulate_austin)

<module 'util.simulate_austin' from 'd:\\Thesis\\med_study_MC\\util\\simulate_austin.py'>

In [4]:
x = simulate_austin.simulate_variables('a')

## Determine beta for a specific treatment effect

In [15]:
def simulate_exposure(variables, a0, 
        al = np.log(1.25),
        am = np.log(1.5),
        ah = np.log(1.75),
        avh = np.log(2)):
    exponent = a0 + al*variables[:,0] + al*variables[:,1]\
        + am*variables[:,3] + am*variables[:,4]\
            + ah*variables[:,6]+ ah*variables[:,7]\
                + avh*variables[:,9]
    p = 1/(1+np.exp(-exponent))
    z = ss.bernoulli.rvs(p=p, size=1000)
    return z

In [16]:
def prevalence_diff(a0, prevalence, variables):
    z = simulate_exposure(variables, a0)
    return np.sum(z)/len(z)-prevalence

We cannot use optimizer since we draw random variables and have thus a not differentiable function.

In [28]:
alpha_res_ls = []
for _ in range(100):
    f = lambda y: prevalence_diff(y, prevalence=0.1, variables=x)
    alpha_res = so.bisect(f, a=-10, b=10)
    alpha_res_ls.append(alpha_res)
alpha_res_mean = np.mean(alpha_res_ls)
print(alpha_res_mean, f(alpha_res_mean))

-2.720404580453453 0.0010000000000000009


In [46]:
z0 = simulate_exposure(variables=x, a0=-2.5)
np.sum(z0)/len(z0)

0.119

In [ ]:
def determine_alpha(prevalence, variables, epsilon=1e-3):
    """Compute alpha0 such that we get the desired prevalence."""
    a0 = 0
    z = np.zeros(len(variables))
    for _ in range(1000):
        z = simulate_exposure(variables, a0)
        if np.abs(np.sum(z)/len(z)-prevalence)<epsilon: 
            break
        elif np.sum(z)/len(z)>prevalence:
           pass 
        

In [25]:
z = simulate_exposure(0.1, x)
plt.hist(z)

TypeError: 'float' object is not subscriptable